In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

from tensorboardX import SummaryWriter


import pandas as pd
import numpy as np
import re

from time import time
import datetime
import os

import xgboost as xgb
import optuna

C:\Users\hp\Documents\Tensorflow\tune\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
data=pd.read_csv('SolarPrediction.csv')

In [9]:
data.head()

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Data                    32686 non-null  object 
 2   Time                    32686 non-null  object 
 3   Radiation               32686 non-null  float64
 4   Temperature             32686 non-null  int64  
 5   Pressure                32686 non-null  float64
 6   Humidity                32686 non-null  int64  
 7   WindDirection(Degrees)  32686 non-null  float64
 8   Speed                   32686 non-null  float64
 9   TimeSunRise             32686 non-null  object 
 10  TimeSunSet              32686 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 2.7+ MB


# DATA ENGINEERING

In [11]:
#Data Feature
data['Month']=data['Data'].apply(lambda x:re.search(r'^\d+',x).group(0)).astype(np.int64)
data['Day']=data['Data'].apply(lambda x:re.search(r'(?<=\/)\d+(?=\/)',x).group(0)).astype(np.int64)
data['Year']=data['Data'].apply(lambda x:re.search(r'(?<=\/)\d+(?=\s)',x).group(0)).astype(np.int64)
data=data.drop('Data',axis=1)

In [12]:
data.head()

,UNIXTime,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year
0,1475229326,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016
1,1475229023,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016
2,1475228726,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016
3,1475228421,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016
4,1475228124,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Time                    32686 non-null  object 
 2   Radiation               32686 non-null  float64
 3   Temperature             32686 non-null  int64  
 4   Pressure                32686 non-null  float64
 5   Humidity                32686 non-null  int64  
 6   WindDirection(Degrees)  32686 non-null  float64
 7   Speed                   32686 non-null  float64
 8   TimeSunRise             32686 non-null  object 
 9   TimeSunSet              32686 non-null  object 
 10  Month                   32686 non-null  int64  
 11  Day                     32686 non-null  int64  
 12  Year                    32686 non-null  int64  
dtypes: float64(4), int64(6), object(3)
memory usage: 3.2+ MB


In [14]:
#Time Feature
data['Hour']=data['Time'].apply(lambda x:re.search(r'(^\d+)',x).group(0)).astype(np.int64)
data['Minute']=data['Time'].apply(lambda x:re.search(r'(?<=\:)\d+(?=\:)',x).group(0)).astype(np.int64)
data['Second']=data['Time'].apply(lambda x:re.search(r'\d+$',x).group(0)).astype(np.int64)
data=data.drop('Time',axis=1)

In [15]:
data.head()

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year,Hour,Minute,Second
0,1475229326,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016,23,55,26
1,1475229023,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016,23,50,23
2,1475228726,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016,23,45,26
3,1475228421,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016,23,40,21
4,1475228124,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016,23,35,24


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Radiation               32686 non-null  float64
 2   Temperature             32686 non-null  int64  
 3   Pressure                32686 non-null  float64
 4   Humidity                32686 non-null  int64  
 5   WindDirection(Degrees)  32686 non-null  float64
 6   Speed                   32686 non-null  float64
 7   TimeSunRise             32686 non-null  object 
 8   TimeSunSet              32686 non-null  object 
 9   Month                   32686 non-null  int64  
 10  Day                     32686 non-null  int64  
 11  Year                    32686 non-null  int64  
 12  Hour                    32686 non-null  int64  
 13  Minute                  32686 non-null  int64  
 14  Second                  32686 non-null

In [17]:
#Sunrise Feature
data['Sunrise_Hour']=data['TimeSunRise'].apply(lambda x:re.search(r'(^\d+)',x).group(0)).astype(np.int64)
data['Sunrise_Minute']=data['TimeSunRise'].apply(lambda x:re.search(r'(?<=\:)\d+(?=\:)',x).group(0)).astype(np.int64)
data=data.drop('TimeSunRise',axis=1)

In [18]:
data.head()


,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunSet,Month,Day,Year,Hour,Minute,Second,Sunrise_Hour,Sunrise_Minute
0,1475229326,1.21,48,30.46,59,177.39,5.62,18:13:00,9,29,2016,23,55,26,6,13
1,1475229023,1.21,48,30.46,58,176.78,3.37,18:13:00,9,29,2016,23,50,23,6,13
2,1475228726,1.23,48,30.46,57,158.75,3.37,18:13:00,9,29,2016,23,45,26,6,13
3,1475228421,1.21,48,30.46,60,137.71,3.37,18:13:00,9,29,2016,23,40,21,6,13
4,1475228124,1.17,48,30.46,62,104.95,5.62,18:13:00,9,29,2016,23,35,24,6,13


In [19]:
#Sunset feature
data['Sunset_Hour']=data['TimeSunSet'].apply(lambda x:re.search(r'(^\d+)',x).group(0)).astype(np.int64)
data['Sunset_Minute']=data['TimeSunSet'].apply(lambda x:re.search(r'(?<=\:)\d+(?=\:)',x).group(0)).astype(np.int64)
data=data.drop('TimeSunSet',axis=1)

In [20]:
data.head()

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Month,Day,Year,Hour,Minute,Second,Sunrise_Hour,Sunrise_Minute,Sunset_Hour,Sunset_Minute
0,1475229326,1.21,48,30.46,59,177.39,5.62,9,29,2016,23,55,26,6,13,18,13
1,1475229023,1.21,48,30.46,58,176.78,3.37,9,29,2016,23,50,23,6,13,18,13
2,1475228726,1.23,48,30.46,57,158.75,3.37,9,29,2016,23,45,26,6,13,18,13
3,1475228421,1.21,48,30.46,60,137.71,3.37,9,29,2016,23,40,21,6,13,18,13
4,1475228124,1.17,48,30.46,62,104.95,5.62,9,29,2016,23,35,24,6,13,18,13


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Radiation               32686 non-null  float64
 2   Temperature             32686 non-null  int64  
 3   Pressure                32686 non-null  float64
 4   Humidity                32686 non-null  int64  
 5   WindDirection(Degrees)  32686 non-null  float64
 6   Speed                   32686 non-null  float64
 7   Month                   32686 non-null  int64  
 8   Day                     32686 non-null  int64  
 9   Year                    32686 non-null  int64  
 10  Hour                    32686 non-null  int64  
 11  Minute                  32686 non-null  int64  
 12  Second                  32686 non-null  int64  
 13  Sunrise_Hour            32686 non-null  int64  
 14  Sunrise_Minute          32686 non-null

In [22]:
data.dtypes

UNIXTime                    int64
Radiation                 float64
Temperature                 int64
Pressure                  float64
Humidity                    int64
WindDirection(Degrees)    float64
Speed                     float64
Month                       int64
Day                         int64
Year                        int64
Hour                        int64
Minute                      int64
Second                      int64
Sunrise_Hour                int64
Sunrise_Minute              int64
Sunset_Hour                 int64
Sunset_Minute               int64
dtype: object

In [23]:
data=data.drop('Year',axis=1)
data.head()

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Month,Day,Hour,Minute,Second,Sunrise_Hour,Sunrise_Minute,Sunset_Hour,Sunset_Minute
0,1475229326,1.21,48,30.46,59,177.39,5.62,9,29,23,55,26,6,13,18,13
1,1475229023,1.21,48,30.46,58,176.78,3.37,9,29,23,50,23,6,13,18,13
2,1475228726,1.23,48,30.46,57,158.75,3.37,9,29,23,45,26,6,13,18,13
3,1475228421,1.21,48,30.46,60,137.71,3.37,9,29,23,40,21,6,13,18,13
4,1475228124,1.17,48,30.46,62,104.95,5.62,9,29,23,35,24,6,13,18,13


In [24]:
data['Sunrise_Hour'].unique()

array([6], dtype=int64)

In [25]:
data=data.drop('Sunrise_Hour',axis=1)

In [27]:
data.to_csv('Engineered.csv',index=False)

# Data Scaling

In [28]:
x=data.drop('Radiation',axis=1).copy()
y=data['Radiation'].copy()

In [29]:
scaler=StandardScaler()


x_data=scaler.fit_transform(x)

In [30]:
pd.DataFrame(x_data)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.937753,-0.500439,0.678974,-0.616253,0.407620,-0.178738,-1.391540,1.512248,1.655482,1.589236,0.660811,-0.943425,1.369126,-1.525765
1,-0.937854,-0.500439,0.678974,-0.654730,0.400285,-0.823359,-1.391540,1.512248,1.655482,1.299687,0.429167,-0.943425,1.369126,-1.525765
2,-0.937953,-0.500439,0.678974,-0.693206,0.183490,-0.823359,-1.391540,1.512248,1.655482,1.010138,0.660811,-0.943425,1.369126,-1.525765
3,-0.938054,-0.500439,0.678974,-0.577776,-0.069497,-0.823359,-1.391540,1.512248,1.655482,0.720589,0.274737,-0.943425,1.369126,-1.525765
4,-0.938153,-0.500439,0.678974,-0.500823,-0.463407,-0.178738,-1.391540,1.512248,1.655482,0.431040,0.506381,-0.943425,1.369126,-1.525765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,0.845373,-1.145490,0.130250,1.038241,0.023209,0.145006,1.344003,-1.701824,-1.672098,-0.437606,-1.037912,0.860876,-0.730393,0.293766
32682,0.845273,-1.145490,-0.052658,1.038241,-0.309138,0.145006,1.344003,-1.701824,-1.672098,-0.727155,-1.269556,0.860876,-0.730393,0.293766
32683,0.845173,-1.145490,-0.052658,1.038241,0.020443,0.789627,1.344003,-1.701824,-1.672098,-1.016704,-1.269556,0.860876,-0.730393,0.293766
32684,0.845073,-1.145490,-0.052658,0.999764,0.248901,0.465884,1.344003,-1.701824,-1.672098,-1.306253,-1.192341,0.860876,-0.730393,0.293766


In [31]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)


x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=100)

In [32]:
print("x_train:",x_train.shape[0])
print('x_test',x_test.shape[0])
print('y_train',y_train.shape[0])
print('y_test',y_test.shape[0])
print('x_val',x_val.shape[0])
print('y_val',y_val.shape[0])

x_train: 18304
x_test 9806
y_train 18304
y_test 9806
x_val 4576
y_val 4576


In [33]:
dtrain=xgb.DMatrix(x_train,label=y_train)
dtest=xgb.DMatrix(x_test,label=y_test)
dval=xgb.DMatrix(x_val,label=y_val)

# Training Model

In [34]:
def rmse_model(params):
    model=xgb.train(params,dtrain,num_boost_round=100,evals=[(dval,'evals')],early_stopping_rounds=10,verbose_eval=0)
    result=model.eval(dval)
    rmse=np.float64(re.search(r"[\d.]+$",result).group(0))
    return rmse


In [29]:
#params={"Learning_rate":0.01,"max_depth":6,"alpha":0.01,"lambda":1.0}
#rmse_model(params)

In [35]:
def optuna_objective(trial):
    learning_rate=trial.suggest_loguniform("learning_rate",0.00001,10.0)
    max_depth=trial.suggest_int('max_depth',4,8)
    l1_reg=trial.suggest_loguniform("l1_reg",0.00001,10.0)
    l2_reg=trial.suggest_loguniform("l2_reg",0.00001,10.0)
    params={"learning_rate":learning_rate,"max_depth":max_depth,"alpha":l1_reg,"lambda":l2_reg}
    
    return rmse_model(params)

In [36]:
class TensorBoardCallback(xgb.callback.TrainingCallback):
    def __init__(self, experiment: str = None, data_name: str = None):
        self.experiment = experiment or "logs"
        self.data_name = data_name or "test"
        self.datetime_ = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        self.log_dir = f"runs/{self.experiment}/{self.datetime_}"
        self.train_writer = SummaryWriter(log_dir=os.path.join(self.log_dir, "train/"))
        if self.data_name:
            self.test_writer = SummaryWriter(
                log_dir=os.path.join(self.log_dir, f"{self.data_name}/")
            )

    def after_iteration(
        self, model, epoch: int, evals_log: xgb.callback.TrainingCallback.EvalsLog
    ) -> bool:
        if not evals_log:
            return False

        for data, metric in evals_log.items():
            for metric_name, log in metric.items():
                score = log[-1][0] if isinstance(log[-1], tuple) else log[-1]
                if data == "train":
                    self.train_writer.add_scalar(metric_name, score, epoch)
                else:
                    self.test_writer.add_scalar(metric_name, score, epoch)

        return False

In [66]:
study=optuna.create_study()
study.optimize(optuna_objective,n_trials=120,show_progress_bar=True)

[I 2023-03-11 06:19:38,456] A new study created in memory with name: no-name-d36c8e0b-1db9-45d5-adf6-394df0711289
C:\Users\hp\Documents\Tensorflow\tune\lib\site-packages\optuna\progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
  0%|                                                                                          | 0/120 [00:00<?, ?it/s]C:\Users\hp\AppData\Local\Temp\ipykernel_6856\2406076344.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate=trial.suggest_loguniform("learning_rate",0.00001,10.0)
C:\Users\hp\AppData\Local\Temp\ipykernel_6856\2406076344.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optun

[I 2023-03-11 06:19:40,956] Trial 0 finished with value: 375.6709549192406 and parameters: {'learning_rate': 2.272147781683205e-05, 'max_depth': 6, 'l1_reg': 3.828043161800561, 'l2_reg': 2.582345561708748}. Best is trial 0 with value: 375.6709549192406.


Best trial: 1. Best value: 371.084:   2%|▊                                             | 2/120 [00:04<03:57,  2.01s/it]

[I 2023-03-11 06:19:42,628] Trial 1 finished with value: 371.08416423394635 and parameters: {'learning_rate': 0.00016897763530171933, 'max_depth': 4, 'l1_reg': 0.00014450588708801612, 'l2_reg': 0.00012609023303893408}. Best is trial 1 with value: 371.08416423394635.


Best trial: 2. Best value: 337.504:   2%|█▏                                            | 3/120 [00:07<05:25,  2.78s/it]

...

[I 2023-03-11 06:24:52,597] Trial 117 finished with value: 76.08941760422344 and parameters: {'learning_rate': 0.27163015706828, 'max_depth': 8, 'l1_reg': 9.897630073067097, 'l2_reg': 0.00018816203823786136}. Best is trial 81 with value: 74.43404976813486.


Best trial: 81. Best value: 74.434:  99%|███████████████████████████████████████████▋| 119/120 [05:17<00:02,  2.56s/it]

[I 2023-03-11 06:24:56,338] Trial 118 finished with value: 77.25986568243492 and parameters: {'learning_rate': 0.04616892457991944, 'max_depth': 8, 'l1_reg': 0.7581570041398039, 'l2_reg': 0.0002574593071725132}. Best is trial 81 with value: 74.43404976813486.


Best trial: 81. Best value: 74.434: 100%|████████████████████████████████████████████| 120/120 [05:21<00:00,  2.68s/it]

[I 2023-03-11 06:25:00,140] Trial 119 finished with value: 85.6054425284863 and parameters: {'learning_rate': 0.026078479052923022, 'max_depth': 8, 'l1_reg': 1.6685324804951336, 'l2_reg': 8.963564807814304e-05}. Best is trial 81 with value: 74.43404976813486.


In [68]:
best_parameters=study.best_params

In [69]:
best_parameters

{'learning_rate': 0.13270238959500247,
 'max_depth': 8,
 'l1_reg': 3.0361338078032114,
 'l2_reg': 0.00010638966488047717}

In [70]:

model=xgb.train(best_parameters,dtrain,num_boost_round=10000,evals=[(dtrain,'train'),(dval,'evals')],early_stopping_rounds=10,
                callbacks=[TensorBoardCallback(experiment='exp_3', data_name='Validation')])

#model=xgb.train(best_parameters,dtrain,num_boost_round=10000,evals=[(dtrain,'train'),(dval,'evals')],early_stopping_rounds=10,
                #callbacks=[tb_callback])

[06:29:15] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "l1_reg", "l2_reg" } are not used.

[0]	train-rmse:332.54634	evals-rmse:330.90083
[1]	train-rmse:293.02819	evals-rmse:291.65454
[2]	train-rmse:259.13428	evals-rmse:258.07829
[3]	train-rmse:230.05055	evals-rmse:229.16060
[4]	train-rmse:205.02773	evals-rmse:204.32791
[5]	train-rmse:183.46610	evals-rmse:182.95693
[6]	train-rmse:165.08805	evals-rmse:165.18868
[7]	train-rmse:149.51449	evals-rmse:150.17125
[8]	train-rmse:136.29278	evals-rmse:137.54469
[9]	train-rmse:124.76247	evals-rmse:126.82642
[10]	train-rmse:115.36404	evals-rmse:118.21530
[11]	train-rmse:107.25696	evals-rmse:111.09596
[12]	train-rmse:100.62284	evals-rmse:105.18209
[13]	train-rmse:94.70202	evals-rmse:100.06774
[14]	train-rmse:89.97935	evals-rmse:96.12188
[15]	train-rmse:85.98321	evals-rmse:92.87527
[16]	train-rmse:82.61844	evals-rmse:90.30803
[17]	train-r

# Results

In [36]:
y_test

22107     675.86
3780     1286.85
13651       1.23
30473       4.43
9084        1.22
          ...   
30536       1.28
14613       5.62
3288       18.25
29353       1.27
8074       73.52
Name: Radiation, Length: 9806, dtype: float64

In [71]:
y_true=np.array(y_test,dtype=np.float64)
y_pred=np.array(model.predict(dtest),dtype=np.float64)

In [60]:
y_true

array([6.75860e+02, 1.28685e+03, 1.23000e+00, ..., 1.82500e+01,
       1.27000e+00, 7.35200e+01])

In [61]:
y_pred

array([ 608.94445801, 1047.39562988,    2.0612011 , ...,   22.53059196,
          1.80770397,   68.5411911 ])

In [72]:
r2=r2_score(y_true,y_pred)
print("r2 score:{:.2f}".format(r2))

r2 score:0.93


In [64]:
dataframe=pd.DataFrame(y_true,columns=['Test Value'])
dataframe['Predicted value']=pd.DataFrame(y_pred,columns=['Predicted value'])

In [65]:
dataframe

,Test Value,Predicted value
0,675.86,608.944458
1,1286.85,1047.395630
2,1.23,2.061201
3,4.43,3.795268
4,1.22,1.370586
...,...,...
9801,1.28,-0.944647
9802,5.62,7.817568
9803,18.25,22.530592
9804,1.27,1.807704
